In [1]:
from pathlib import Path
import pydicom as dcm
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os 

#____________________
stat_data=pd.read_csv(r"C:\Users\Hiran Gunawardana\Desktop\KYS Project\stat_data_for_algorithm.csv")
organs_of_interest = "Heart Breast_L Breast_R Lung_L Lung_R CTV PTV"
OOI=organs_of_interest.split(" ")

#____________________

#Define patient file directory
#------------------------------

directory = r'C:\Users\Hiran Gunawardana\Desktop\KYS Project\NEW'

#------------------------------

data_dict = {}
result_dict = {}
x_diff_CTV = x_diff_PTV = 0

for foldername in os.listdir(directory):
    folderpath = os.path.join(directory, foldername)
    if os.path.isdir(folderpath):
        for filename in os.listdir(folderpath):
            filepath = os.path.join(folderpath, filename)
            if os.path.isfile(filepath) and filepath.endswith('.dcm'):
                dcm_file = dcm.dcmread(filepath)
                if dcm_file["Modality"][0:] == "RTSTRUCT":
                    
                    # Proceeding only the DICOM file is from the RTSTRUCT modality
                    #------------------------------------------------------
                    
                    contours=dcm_file.ROIContourSequence
                    organs=dcm_file.StructureSetROISequence
                    
                    #Extracting the organ data (Coordinates of all the slices) from the RTSTRUCT file.
                    
                    organ_data={}
                    
                    for j in range(len(contours)):
                        contour_data = np.array([])
                        #for loop to get the cordinates from all the slices of of the organ
                        for i in range(len(contours[j].ContourSequence)):
                            contour_data=np.concatenate((contour_data,(np.array(contours[j].ContourSequence[i].ContourData))))
                            #getting all the cordinate data to a data frame. with x,y,z cordinates
                            organ_data[f"organdata{j}"]=pd.DataFrame(contour_data.reshape(int(len(contour_data)/3),3),columns=("X","Y","Z"))
                    if foldername not in data_dict:
                        data_dict[foldername] = []
                    
                    # Calculating organ volume form the slice data
                    
                    organ_volume = {}
                    for k in range(len(contours)):
                        contour_areas = []
                        z_coords = organ_data[f"organdata{k}"]["Z"].unique()
                        for z in z_coords:
                            # get the contour data for this slice
                            slice_contour = organ_data[f"organdata{k}"][organ_data[f"organdata{k}"]["Z"] == z][["X", "Y"]].values

                            # calculate the area of the contour in this slice
                            contour_area = 0.5 * np.abs(np.dot(slice_contour[:, 0], np.roll(slice_contour[:, 1], 1)) - np.dot(slice_contour[:, 1], np.roll(slice_contour[:, 0], 1)))
                            contour_areas.append(contour_area)

                        # calculate the organ volume using slice thickness and contour areas
                        slice_thickness = np.abs(organ_data[f"organdata{k}"]["Z"].max() - organ_data[f"organdata{k}"]["Z"].min()) / (len(z_coords) - 1)
                        organ_volume[f"organvolume{k}"] = np.sum(contour_areas) * slice_thickness
                                        
                    # to find the contour number in which the body contour is located
                    # Used to calculate the relative locations of the organs with respect to the size of the body.
                    
                    for i in range(len(contours)):
                        if 'Body' in organs[i].ROIName:
                            body=int(i)
                        elif 'Breast_R' in organs[i].ROIName:
                            breast_R=int(i)    
                            
                    #Calculate the body axis to determine relative position of the organs in each side of the body
                    
                    body_x_max,body_x_min = organ_data[f"organdata{body}"]["X"].max(),organ_data[f"organdata{body}"]["X"].min()
                    body_y_max,body_y_min = organ_data[f"organdata{body}"]["Y"].max(),organ_data[f"organdata{body}"]["Y"].min()
                    body_axis_x=(body_x_max+body_x_min)/2
                    body_axis_y=(body_y_max+body_y_min)/2

                    anthrop_data = pd.DataFrame( {
                        "organ_number": range(len(organs)),
                        "organ_name": [organ.ROIName for organ in organs],
                        "organ_volume": [round(organ_volume[f"organvolume{i}"] / 1e6, 2) for i in range(len(organs))],
                        "organ_axis_x": [((organ_data[f"organdata{i}"]["X"].max()+organ_data[f"organdata{i}"]["X"].min())/2-body_axis_x) for i in range(len(organs))],
                        "organ_axis_y": [((organ_data[f"organdata{i}"]["Y"].max()+organ_data[f"organdata{i}"]["Y"].min())/2-body_axis_y) for i in range(len(organs))],
                    })
                                
                    
                    anthrop_data["%_dist_from_x_axis"]=anthrop_data["organ_axis_x"]*100/((body_x_max-body_x_min)/2)
                    anthrop_data["%_dist_from_y_axis"]=anthrop_data["organ_axis_y"]*100/((body_y_max-body_y_min)/2)

                    #data_dict[foldername].append(anthrop_data)

                    AD=anthrop_data
                    
                    # OAD = Organ Axis Data
                    # Determining organ axis data for each interested organ.
                    
                    OAD={}
                    for organ in OOI:
                        if organ in stat_data:
                            OAD[f"{organ}_x_up"] = (stat_data[f"{organ}_%_dist_x"][1] + (3*stat_data[f"{organ}_%_dist_x"][2]))
                            OAD[f"{organ}_x_low"] = (stat_data[f"{organ}_%_dist_x"][1] - (3*stat_data[f"{organ}_%_dist_x"][2]))
                            OAD[f"{organ}_y_up"] = (stat_data[f"{organ}_%_dist_y"][1] + (3*stat_data[f"{organ}_%_dist_y"][2]))
                            OAD[f"{organ}_y_low"] = (stat_data[f"{organ}_%_dist_y"][1] - (3*stat_data[f"{organ}_%_dist_y"][2]))

                            OAD[f"{organ}_vol_up"] = (stat_data[f"{organ}"][1] + (3*stat_data[f"{organ}"][2]))
                            OAD[f"{organ}_vol_low"] = (stat_data[f"{organ}"][1] - (3*stat_data[f"{organ}"][2]))
                    
                    # Defining a function to determine the organ name via the algorithm.
                    def organname(i):
                        organ_volume = AD["organ_volume"][int(f"{i}")]
                        max_organ_volume = AD["organ_volume"].max()

                        if (OAD['Heart_x_low'] < AD["%_dist_from_x_axis"][int(f"{i}")] < OAD['Heart_x_up']) and (OAD['Heart_vol_low'] < organ_volume < OAD['Heart_vol_up']):
                            return "Heart"
                        elif (OAD['Breast_R_x_low'] < AD["%_dist_from_x_axis"][int(f"{i}")] < OAD['Breast_R_x_up']) and (OAD['Breast_R_vol_low'] < organ_volume < OAD['Breast_R_vol_up']):
                            return "Breast_R"
                        elif (OAD['Lung_L_x_low'] < AD["%_dist_from_x_axis"][int(f"{i}")] < OAD['Lung_L_x_up']) and (OAD['Lung_L_vol_low'] < organ_volume < OAD['Lung_L_vol_up']):
                            return "Lung_L"
                        elif (OAD['Lung_R_x_low'] < AD["%_dist_from_x_axis"][int(f"{i}")] < OAD['Lung_R_x_up']) and (OAD['Lung_R_vol_low'] < organ_volume < OAD['Lung_R_vol_up']):
                            return "Lung_R"
                        elif organ_volume == max_organ_volume:
                            return "Body"
                        else:
                            return identify_PTV(i)
                    
                    #identifying CTV
                    def identify_CTV(x):   
                                              
                        for p in range(len(organs)):
                            organ_x1 = organ_data[f"organdata{p}"]["X"][organ_data[f"organdata{p}"]["Z"] == breast_z1]
                            organ_x2 = organ_data[f"organdata{p}"]["X"][organ_data[f"organdata{p}"]["Z"] == breast_z2]
                            organ_x3 = organ_data[f"organdata{p}"]["X"][organ_data[f"organdata{p}"]["Z"] == breast_z3]
                            
                            
                            x_diff_CTV = (body_x1.max()+body_x2.max()+body_x3.max()) - (organ_x1.max()+organ_x2.max()+organ_x3.max())
                            
                            x_diff[p] = x_diff_CTV
                            x_diff[body] = 1000
                            x_diff[PTV_organ_num] = 1000
                            

                        CTV_x_diff_min = min([value for value in x_diff.values() if not np.isnan(value)])
                        CTV_organ_num = list(x_diff.keys())[list(x_diff.values()).index(CTV_x_diff_min)]

                        if CTV_organ_num == x and (0.3<(AD["organ_volume"][x]/AD["organ_volume"][Breast_R])<1.5):
                            return 'CTV'
                        else:
                            return "---"
                    
                    x_diff = {}
                    
                    #identifying PTV
                    
                    def identify_PTV(x):
                        global breast_z1, breast_z2, breast_z3                      
                        global body_x1, body_x2, body_x3
                        
                        global PTV_organ_num  # Declare PTV_organ_num as a global variable
                        global PTV_x_diff_min
                        global Breast_R
                        
                        i = 0
                        for organ in organs:
                            if organ.ROIName == "Breast_R":
                                Breast_R = i
                            else:
                                i += 1

                        breast_z1 = organ_data[f"organdata{Breast_R}"]["Z"].unique()[int(len(organ_data[f"organdata{Breast_R}"]["Z"].unique()) / 3)]
                        breast_z2 = organ_data[f"organdata{Breast_R}"]["Z"].unique()[int(len(organ_data[f"organdata{Breast_R}"]["Z"].unique()) *2 / 3)]
                        breast_z3 = organ_data[f"organdata{Breast_R}"]["Z"].unique()[int(len(organ_data[f"organdata{Breast_R}"]["Z"].unique()) -1 )]
                        
                        body_x1 = organ_data[f"organdata{body}"]["X"][organ_data[f"organdata{body}"]["Z"] == breast_z1]
                        body_x2 = organ_data[f"organdata{body}"]["X"][organ_data[f"organdata{body}"]["Z"] == breast_z2]
                        body_x3 = organ_data[f"organdata{body}"]["X"][organ_data[f"organdata{body}"]["Z"] == breast_z3]
                        
                        for p in range(len(organs)):
                            organ_x1 = organ_data[f"organdata{p}"]["X"][organ_data[f"organdata{p}"]["Z"] == breast_z1]
                            organ_x2 = organ_data[f"organdata{p}"]["X"][organ_data[f"organdata{p}"]["Z"] == breast_z2]
                            organ_x3 = organ_data[f"organdata{p}"]["X"][organ_data[f"organdata{p}"]["Z"] == breast_z3]
                            
                            x_diff_PTV = (body_x1.max()+body_x2.max()+body_x3.max()) - (organ_x1.max()+organ_x2.max()+organ_x3.max())
                            
                            x_diff[p] = x_diff_PTV
                            x_diff[body] = 1000

                        PTV_x_diff_min = min([value for value in x_diff.values() if not np.isnan(value)])
                        PTV_organ_num = list(x_diff.keys())[list(x_diff.values()).index(PTV_x_diff_min)]
                        
                        if PTV_organ_num == x :
                            return 'PTV'                        
                        
                        else:
                            return identify_CTV(x)
                    
                    # Returning the result from running the algorithm on provided DICOM files.
                    algorithm = pd.DataFrame({
                    
                    "organ_number": range(len(organs)),
                    "organ_name": [organ.ROIName for organ in organs],
                    "organ_identified": [organname(i) for i in range(len(organs))],
                    
                    })

                    algorithm["test"] = algorithm.apply(lambda row: "Pass" if ((row["organ_identified"] in row["organ_name"]) or (row["organ_name"] in row["organ_identified"]) ) else "Fail", axis=1)
                  
                    if foldername not in result_dict:
                        result_dict[foldername] = []
                    result_dict[foldername].append(algorithm)

                    def results():
                        for key, value in result_dict.items():
                            print(f"PATIENT {key}:")
                            print(value)
                            print()
                            
                    def result_stats():
                        result_stats = result_dict["1"][0][["organ_name", "test"]]

                        for i in range(2, len(result_dict) + 1):
                            result_stats = pd.concat([result_stats, result_dict[str(i)][0][["organ_name", "test"]]], ignore_index=True)

                        organ_value_counts = result_stats["organ_name"].value_counts()
                        organ_pass_counts = result_stats[result_stats["test"]=="Pass"]["organ_name"].value_counts()
                        pass_percentage = organ_pass_counts*100/organ_value_counts
                        
                        return pass_percentage.sort_values(ascending=False)

In [2]:
results()


PATIENT 1:
[   organ_number  organ_name organ_identified  test
0             0        Body             Body  Pass
1             1    Breast_R         Breast_R  Pass
2             2       Heart            Heart  Pass
3             3      Lung_L           Lung_L  Pass
4             4      Lung_R           Lung_R  Pass
5             5  PTV_InMarg              ---  Fail]

PATIENT 10:
[   organ_number  organ_name organ_identified  test
0             0        Body             Body  Pass
1             1    Breast_R         Breast_R  Pass
2             2       Heart            Heart  Pass
3             3      Lung_L           Lung_L  Pass
4             4      Lung_R              ---  Fail
5             5  PTV_InMarg              ---  Fail]

PATIENT 100:
[   organ_number  organ_name organ_identified  test
0             0        Body             Body  Pass
1             1    Breast_R         Breast_R  Pass
2             2       Heart            Heart  Pass
3             3      Lung_L           L

In [3]:
result_stats()

Body          100.0
Breast_R       94.0
Heart          89.0
Lung_L         82.0
Lung_R         57.0
PTV_InMarg     51.0
BODY            NaN
Name: organ_name, dtype: float64